In [1]:
import numpy as np
import pandas as pd
import scipy as sc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

PATH = "/home/salman/Dropbox/BaharProject/dataset/"
FILE = "cell-ids.npy"

In [2]:
TEST=np.load("%s"%(PATH+FILE))
df=pd.DataFrame.from_records(TEST)

In [3]:
TrshH=0
CUTOFF = 0.6

df.loc[df.label.astype(int)>10,'label'] = 1
df.loc[df.label.astype(int)==2,'label'] = 0

df['ratio'] = 0
df.loc[df.followers > TrshH,'ratio'] = df.loc[df.followers > TrshH,'friends']/df.loc[df.followers > TrshH,'followers']

df['spamLabel'] = 0
df.loc[df.english > CUTOFF,'spamLabel'] = 1

df['tweet'] = df['tweet'].map(lambda a: " ".join(filter(lambda x:x[0]!='@', a.split())))

In [4]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df.tweet)
#X.toarray()

In [5]:
def append_feature_columns(matrix, columns):
    l = len(columns[0])
    for col in columns:
        X = matrix.toarray()
        x = np.array(col).reshape(l, 1)
        X = np.hstack((X, x))

    matrix = sc.sparse.csr.csr_matrix(X)
    return matrix

In [6]:
def classify(X, Y):
    clf = svm.SVC(C=500000)
    #scoring = ['precision_macro', 'recall_macro']
    scores = cross_val_score(clf, X, Y, scoring=scorer, cv=10)
    print(scores)

In [7]:
def getScores(estimator, x, y):
    yPred = estimator.predict(x)
    print(yPred)
    return (accuracy_score(y, yPred), 
            precision_score(y, yPred, average='macro'),
            recall_score(y, yPred, average='macro'))

In [8]:
def scorer(estimator, x, y):
    a, p, r = getScores(estimator, x, y)
    print(a, p, r)
    return a+p+r

In [9]:
X = append_feature_columns(X,[df.ratio.tolist()])

In [10]:
classify(X,df["spamLabel"])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8125 0.40625 0.5
[1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]
0.75 0.5897435897435898 0.5897435897435898
[0 0 0 0 0 0 1 1 0 1 0 1 1 1 0]
0.6666666666666666

/home/salman/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/salman/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 0.611111111111111 0.6666666666666666
[0 0 1 1 0 1 0 1 0 1 0 0 1 0 0]
0.5333333333333333 0.4722222222222222 0.45833333333333337
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8 0.4 0.5
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
0.9333333333333333 0.9615384615384616 0.8333333333333333
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
0.8 0.6730769230769231 0.625
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
0.7333333333333333 0.39285714285714285 0.4583333333333333
[1 1 1 0 0 1 0 1 0 0 0 1 0 0 1]
0.7333333333333333 0.7142857142857143 0.8333333333333333
[1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
0.9333333333333333 0.9615384615384616 0.8333333333333333
[1.71875    1.92948718 1.94444444 1.46388889 1.7        2.72820513
 2.09807692 1.58452381 2.28095238 2.72820513]
